In [1]:
import pandas as pd

In [2]:
weather_data_file = '../data/NOAA_weather_data/2005-2015_california_only_all.csv'
fire_data_file = '../data/USDA_wildfire_data/regridded_california_fires.csv'
training_data_file = '../data/training_data/2005-2015_training_data.csv'

# Parallelization options
N_PROCESSES = 15
JOBS_PER_PROCESS = 1

In [109]:
weather = pd.read_csv(weather_data_file)
weather['time'] = pd.to_datetime(weather['time'])
weather['date'] = weather['time'].dt.date
weather['date'] = pd.to_datetime(weather['date'])

fires = pd.read_csv(fire_data_file)
fires['ignition'] = 1
fires['date'] = pd.to_datetime(fires['date'])

In [110]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11978560 entries, 0 to 11978559
Data columns (total 14 columns):
time        datetime64[ns]
lat         float64
lon         float64
air.sfc     float64
air.2m      float64
apcp        float64
crain       float64
rhum.2m     float64
dpt.2m      float64
pres.sfc    float64
uwnd.10m    float64
vwnd.10m    float64
veg         float64
date        datetime64[ns]
dtypes: datetime64[ns](2), float64(12)
memory usage: 1.2 GB


In [111]:
fires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189550 entries, 0 to 189549
Data columns (total 7 columns):
lat           189550 non-null float64
lon           189550 non-null float64
date          189550 non-null datetime64[ns]
time          110217 non-null float64
size          189550 non-null float64
size_class    189550 non-null object
ignition      189550 non-null int64
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 10.1+ MB


In [124]:
training_data = pd.merge(weather, fires, on=['lat', 'lon', 'date'], how='left')

In [125]:
training_data.head()

,time_x,lat,lon,air.sfc,air.2m,apcp,crain,rhum.2m,dpt.2m,pres.sfc,uwnd.10m,vwnd.10m,veg,date,time_y,size,size_class,ignition
0,2005-01-01,40.29749,-124.3408,284.87308,283.2486,1.406250,1.0,77.47234,279.43463,98986.18,5.984521,3.811233,0.0,2005-01-01,NaN,NaN,NaN,NaN
1,2005-01-01,38.96153,-123.5579,279.62308,279.4986,2.968750,1.0,93.15984,278.41120,98086.18,5.476708,4.858109,73.1,2005-01-01,NaN,NaN,NaN,NaN
2,2005-01-01,39.24150,-123.6393,280.68558,281.1236,5.023438,1.0,82.22234,278.27057,98486.18,4.375146,2.998733,76.0,2005-01-01,NaN,NaN,NaN,NaN
3,2005-01-01,39.52163,-123.7215,280.68558,281.1236,6.937500,1.0,82.22234,278.27057,98386.18,6.336083,4.592484,76.0,2005-01-01,NaN,NaN,NaN,NaN
4,2005-01-01,39.80193,-123.8045,277.49808,277.4361,4.015625,0.0,92.84734,276.31744,95386.18,5.093896,3.061233,74.4,2005-01-01,NaN,NaN,NaN,NaN


In [126]:
training_data['air.sfc'].isnull().sum()

0

In [127]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12085904 entries, 0 to 12085903
Data columns (total 18 columns):
time_x        datetime64[ns]
lat           float64
lon           float64
air.sfc       float64
air.2m        float64
apcp          float64
crain         float64
rhum.2m       float64
dpt.2m        float64
pres.sfc      float64
uwnd.10m      float64
vwnd.10m      float64
veg           float64
date          datetime64[ns]
time_y        float64
size          float64
size_class    object
ignition      float64
dtypes: datetime64[ns](2), float64(15), object(1)
memory usage: 1.7+ GB


In [128]:
training_data.rename(columns={'time_x':'weather_bin_time','time_y':'fire_discovery_time'}, inplace=True)
training_data.drop(['date'], axis=1, inplace=True)
training_data.head()

,weather_bin_time,lat,lon,air.sfc,air.2m,apcp,crain,rhum.2m,dpt.2m,pres.sfc,uwnd.10m,vwnd.10m,veg,fire_discovery_time,size,size_class,ignition
0,2005-01-01,40.29749,-124.3408,284.87308,283.2486,1.406250,1.0,77.47234,279.43463,98986.18,5.984521,3.811233,0.0,NaN,NaN,NaN,NaN
1,2005-01-01,38.96153,-123.5579,279.62308,279.4986,2.968750,1.0,93.15984,278.41120,98086.18,5.476708,4.858109,73.1,NaN,NaN,NaN,NaN
2,2005-01-01,39.24150,-123.6393,280.68558,281.1236,5.023438,1.0,82.22234,278.27057,98486.18,4.375146,2.998733,76.0,NaN,NaN,NaN,NaN
3,2005-01-01,39.52163,-123.7215,280.68558,281.1236,6.937500,1.0,82.22234,278.27057,98386.18,6.336083,4.592484,76.0,NaN,NaN,NaN,NaN
4,2005-01-01,39.80193,-123.8045,277.49808,277.4361,4.015625,0.0,92.84734,276.31744,95386.18,5.093896,3.061233,74.4,NaN,NaN,NaN,NaN


In [129]:
training_data.describe()

,lat,lon,air.sfc,air.2m,apcp,crain,rhum.2m,dpt.2m,pres.sfc,uwnd.10m,vwnd.10m,veg,fire_discovery_time,size,ignition
count,1.208590e+07,1.208590e+07,1.208590e+07,1.208590e+07,1.208590e+07,1.208590e+07,1.208590e+07,1.208590e+07,1.208590e+07,1.208590e+07,1.208590e+07,1.208590e+07,390304.000000,535208.000000,535208.0
mean,3.712559e+01,-1.195266e+02,2.890160e+02,2.880038e+02,1.611407e-01,5.760115e-02,4.780529e+01,2.741876e+02,9.153922e+04,1.060319e+00,1.016114e-01,3.073957e+01,1388.129745,77.409296,1.0
std,2.557480e+00,2.504387e+00,1.392884e+01,1.023288e+01,9.318510e-01,2.329877e-01,2.637219e+01,7.587704e+00,6.960172e+03,2.730290e+00,3.306464e+00,2.275174e+01,492.253793,2260.694785,0.0
min,3.268389e+01,-1.243408e+02,2.422595e+02,2.465075e+02,0.000000e+00,0.000000e+00,1.000000e+00,1.921768e+02,6.624355e+04,-2.030144e+01,-2.000592e+01,0.000000e+00,0.000000,0.001000,1.0
25%,3.499029e+01,-1.214709e+02,2.789076e+02,2.808344e+02,0.000000e+00,0.000000e+00,2.520942e+01,2.691997e+02,8.716753e+04,-7.417145e-01,-2.105519e+00,8.200000e+00,1124.000000,0.100000,1.0
50%,3.683767e+01,-1.199051e+02,2.866094e+02,2.870417e+02,0.000000e+00,0.000000e+00,4.371313e+01,2.747059e+02,9.234483e+04,1.040731e+00,6.075287e-02,3.060000e+01,1429.000000,0.250000,1.0
75%,3.925970e+01,-1.174117e+02,2.974965e+02,2.948267e+02,0.000000e+00,0.000000e+00,6.956379e+01,2.798197e+02,9.708413e+04,2.749374e+00,2.247296e+00,4.360000e+01,1718.000000,1.000000,1.0
max,4.197182e+01,-1.143645e+02,3.384141e+02,3.229702e+02,4.148438e+01,1.000000e+00,1.000312e+02,2.993494e+02,1.035852e+05,2.156605e+01,2.987953e+01,8.370000e+01,2359.000000,255858.000000,1.0


In [130]:
training_data.to_csv(training_data_file, index=False)